In [22]:
!pip install --upgrade google-cloud-aiplatform
!pip install ipywidgets --quiet
!pip install google-cloud-dialogflow-cx



In [ ]:
import os
import uuid
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.cloud.dialogflowcx_v3.types import TextInput, QueryInput
from google.cloud.dialogflowcx_v3.services.sessions import SessionsClient



In [24]:
# 🔑 Path to your JSON key
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/johnwatson/Documents/A Grad School/Hackathon/Clarkie/hackathon-454621-40c3702ee8eb.json"

PROJECT_ID = "hackathon-454621"
LOCATION   = "us-central1"
AGENT_ID   = "eba4b842-c371-47f7-b43e-3f7a710aa927"
SESSION_ID = str(uuid.uuid4())

# Specify the correct regional endpoint
client = SessionsClient(client_options={"api_endpoint": f"{LOCATION}-dialogflow.googleapis.com"})
session = client.session_path(PROJECT_ID, LOCATION, AGENT_ID, SESSION_ID)

def detect_intent_cx(text: str, language_code: str = "en"):
    qi = QueryInput(text=TextInput(text=text), language_code=language_code)
    response = client.detect_intent(request={"session": session, "query_input": qi})
    # Extract text from response messages
    if response.query_result.response_messages:
        for message in response.query_result.response_messages:
            if message.text:
                return message.text.text[0]  # Return the first text response
    return "No response from agent"


In [25]:
import ipywidgets as widgets
from IPython.display import display, clear_output

user_box = widgets.Text(
    placeholder="Type here and press Enter →",
    description="You →",
    layout=widgets.Layout(width="100%")
)
chat_output = widgets.Output(layout=widgets.Layout(border="1px solid #ccc"))

def on_submit(change):
    with chat_output:
        clear_output(wait=True)
        msg = change.value.strip()
        if not msg:
            return
        print(f"You: {msg}")
        reply = detect_intent_cx(msg)
        print(f"Bot: {reply}\n")
        user_box.value = ""

user_box.on_submit(on_submit)
display(user_box, chat_output)


/var/folders/29/6zxrb1cx1k92z7w2f5cy2nd80000gn/T/ipykernel_91477/2088068905.py:22: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  user_box.on_submit(on_submit)


Text(value='', description='You →', layout=Layout(width='100%'), placeholder='Type here and press Enter →')

Output(layout=Layout(border_bottom='1px solid #ccc', border_left='1px solid #ccc', border_right='1px solid #cc…